In [2]:
import os
import time
import datetime
import multiprocessing
# import pymysql
# import itertools

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
os.environ['WDM_LOG_LEVEL'] = '0'
YOUTUBE_CHANNEL = '날림v'

In [3]:
def init_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    # options.add_argument("headless")
    options.add_argument("--log-level=3")
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1280, 1024)
    return driver

In [4]:
def infinite_scroll(driver, SCROLL_PAUSE_TIME=2):
   time.sleep(SCROLL_PAUSE_TIME)

   while True:
      last_height = driver.execute_script("return document.documentElement.scrollHeight")
      driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

      time.sleep(SCROLL_PAUSE_TIME)

      new_height = driver.execute_script("return document.documentElement.scrollHeight")
      if new_height == last_height:
         break
      last_height = new_height

In [5]:
def get_BGM_lists(YOUTUBE_CHANNEL):
    driver = init_driver()
    driver.get(f'https://www.youtube.com/c/{YOUTUBE_CHANNEL}/playlists?view=1&sort=dd&shelf_id=0')
    infinite_scroll(driver)

    playlists = driver.find_elements(By.CSS_SELECTOR, '#items > ytd-grid-playlist-renderer')
    BGM_lists = [x.find_element(By.CSS_SELECTOR, '#video-title') for x in playlists
                if any(word in x.find_element(By.CSS_SELECTOR, '#video-title').text for word in ['BGM'])]

    playlist_href = [x.get_attribute("href").split('=')[-1] for x in BGM_lists]
    playlist_title = [x.text for x in BGM_lists]
    BGM_lists = sorted(dict(zip(playlist_title, playlist_href)).items())
    driver.quit()

    return BGM_lists

In [6]:
BGM_lists = get_BGM_lists(YOUTUBE_CHANNEL)
print(BGM_lists)

[('BGM 2020 07', 'PLj63tGIo1sSq_E5c9ptp8xPBcS6tANq18'), ('BGM 2020 08', 'PLj63tGIo1sSqZgEo3it5d7hnVo3lIgKRp'), ('BGM 2020 09', 'PLj63tGIo1sSphQ5n5OEL2m_EzISuNwCpI'), ('BGM 2020 10', 'PLj63tGIo1sSr17V1Rme01a18KXvHDOyRk'), ('BGM 2020 11', 'PLj63tGIo1sSrymhqcZjGN5RpkXcx2-Qse'), ('BGM 2020 12', 'PLj63tGIo1sSpUZJeAAAl30dsUmyJQvQ7J'), ('BGM 2021 01', 'PLj63tGIo1sSo9GtiWnSNxr5PNCsRSgqFX'), ('BGM 2021 02', 'PLj63tGIo1sSq9NXV8qpkzQaSFLXxK9Pji'), ('BGM 2021 03', 'PLj63tGIo1sSrKSDvgeRhP3WJa7yszI247'), ('BGM 2021 04', 'PLj63tGIo1sSqmoXWRh96aMgulu-VR0qzr'), ('BGM 2021 05', 'PLj63tGIo1sSqYOG0UIx6G5BbJmDKl1TXx'), ('BGM 2021 06', 'PLj63tGIo1sSqievrWIwa7k5VvcGMCatld'), ('BGM 2021 07', 'PLj63tGIo1sSpDwSJxMGSED_gA1KVr8PdT'), ('BGM 2021 08', 'PLj63tGIo1sSra3EYTtpean0kw9XOdJzd4'), ('BGM 2021 09', 'PLj63tGIo1sSoJUOUTqnhpKryf87f1034o'), ('BGM 2021 10', 'PLj63tGIo1sSrAijwosgz5PNrY6yQnqwkR'), ('BGM 2021 11', 'PLj63tGIo1sSpmoVd1qrJMdL_qGWymbCw0'), ('BGM 2021 12', 'PLj63tGIo1sSqrSt2TnyOgiJbtb1Tb0K0a')]


In [7]:
def get_list_items(BGM_list):
    total = []
    driver = init_driver()
    driver.get('https://www.youtube.com/playlist?list='+BGM_list[1])
    infinite_scroll(driver)

    videos = driver.find_elements(By.CSS_SELECTOR, '#meta')[:-1]
    rows = [(x.text.split("\n")[0],
            x.text.split("\n")[1],
            x.find_element(By.CSS_SELECTOR, '#video-title').get_attribute("href").split('&')[0].split('=')[1])
            for x in videos]
    total.extend(rows)
    driver.quit()
    return total

In [8]:
p = multiprocessing.Pool(5)
start = time.time()
total_lists = []
temp = p.apply_async(get_list_items, (BGM_lists[0],))
print(temp.get())
p.close()
p.join()
# map(lambda x: x.get(), total_lists)
# print(total_lists)

In [ ]:
videos = driver.find_elements(By.CSS_SELECTOR, '#meta')[:-1]
print([x.find_element(By.CSS_SELECTOR, '#video-title').get_attribute("href").split('&')[0].split('=')[1]
        for x in videos])